7-8

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", version = 1, cache=True, as_frame=False)  #pandasのデータフレームにならないように設定
X, y = mnist["data"], mnist["target"]
X_train, X_valid, X_test, y_train, y_valid, y_test = X[:50000], X[50000:60000], X[60000:], y[:50000], y[50000:60000], y[60000:]

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
#ハード投票
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

ext_clf = ExtraTreesClassifier()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

hard_voting_clf = VotingClassifier(
    estimators=[("ext", ext_clf), ("rf", rnd_clf), ("svc", svm_clf)],
    voting="hard"
)

print("検証セットでの各々の正解率は以下。")
for clf in (ext_clf, rnd_clf, svm_clf, hard_voting_clf):
    clf.fit(X_train, y_train)
    y_pred_valid = clf.predict(X_valid)
    print(clf.__class__.__name__, accuracy_score(y_valid, y_pred_valid))

hard_voting_clf.fit(X_train, y_train)
y_pred_test = hard_voting_clf.predict(X_test)
print(f"ハード投票分類器の正解率は{accuracy_score(y_test, y_pred_test)}")


検証セットでの各々の正解率は以下。
ExtraTreesClassifier 0.9738
RandomForestClassifier 0.972
SVC 0.9802
VotingClassifier 0.9778
ハード投票分類器の正解率は0.9728


In [3]:
#ソフト投票
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

ext_clf = ExtraTreesClassifier()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

soft_voting_clf = VotingClassifier(
    estimators=[("ext", ext_clf), ("rf", rnd_clf), ("svc", svm_clf)],
    voting="soft"
)

soft_voting_clf.fit(X_train, y_train)
y_pred_valid = soft_voting_clf.predict(X_valid)
print(f"検証セットでの正解率は{accuracy_score(y_valid, y_pred_valid)}")
y_pred_test = soft_voting_clf.predict(X_test)
print(f"ソフト投票分類器の正解率は{accuracy_score(y_test, y_pred_test)}")


検証セットでの正解率は0.9816
ソフト投票分類器の正解率は0.9785


7-9

In [6]:
estimators = [ext_clf, rnd_clf, svm_clf]
X_valid_predictions = np.empty((len(X_valid), len(estimators)), dtype=np.float32)

ext_clf.fit(X_train, y_train)
rnd_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)
X_valid_predictions[:, 0] = ext_clf.predict(X_valid)
X_valid_predictions[:, 1] = rnd_clf.predict(X_valid)
X_valid_predictions[:, 2] = svm_clf.predict(X_valid)
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_valid_predictions, y_valid)
rnd_forest_blender.oob_score_

0.9783

In [7]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

y_pred = rnd_forest_blender.predict(X_test_predictions)

In [8]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9756